In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams.update({'font.size': 12})

In [ ]:
base = '/users/jmy/data/image_sound'
sbjs_num = 15
sbj_list = [25,26,29,30,31,32,33,34,37,38,39,40,41,43,44]
interest_list = ['Digit','Object','Vs','Magnitude','Animacy']
stim_type_list = ['etr0']#'etr'

In [ ]:
# RDM codes
dgt_cd = np.zeros((16,16))
dgt_cd[:10,:10] = 1
obj_cd = np.zeros((16,16))
obj_cd[10:,10:] = 1
vs_cd = np.zeros((16,16))
vs_cd[:10,10:] = 1
vs_cd[10:,:10] = 1
mag_cd = np.zeros((16,16))
mag_cd[7:10,1:4] = 1
ani_cd = np.zeros((16,16))
ani_cd[13:,10:13] = 1
cds_list = [dgt_cd,obj_cd,vs_cd,ani_cd,mag_cd]

In [ ]:
# Preparing plot
sns.set(style='dark',font_scale=1.3)
col_list = sns.color_palette('Spectral_r',15)
col_arr = np.array(col_list)
xticks = np.arange(15)
xlabels=['1','2','3','4','5','6','7','8','9','10','11','12','13','Fc','Out\nput']
tt_list = ['Digit','Object','Digit vs. Object','Magnitude','Animacy']

In [ ]:
def rdm_reord(org):
    new = org.copy()
    new[10:15,10:15] = org[11:,11:]
    new[15,15] = org[10,10]
    new[-1,10:15] = org[10,11:]
    new[10:15,-1] = org[11:,10]
    return new

def plot_corr_bar(grp,ylim=None,ysp=None,capsize=0,corr_type='Spear',std=False):
    grp_mean = np.mean(grp,axis=0)
    if std:
        grp_sted = np.std(grp,axis=0)
    else:
        grp_sted = np.std(grp,axis=0) / (15**0.5)
    f, axes = plt.subplots(1,1,figsize=(6,4),facecolor='white',constrained_layout=True)
    axes.bar(xticks,grp_mean,color=col_arr,width=0.9,
                yerr=grp_sted,error_kw=dict(lw=1.2),ecolor='k',linewidth=1.5,capsize=capsize)
    axes.set_xticks(xticks)
    axes.set_xticklabels(xlabels)
    if ylim != None:
        axes.set_yticks(np.arange(ylim[0],ylim[1]+0.1,ysp))
    if corr_type == 'Spear':
        ylabel = "Spearman's "+r"$\rho$"
    elif corr_type == 'None':
        ylabel = "Similarity score"
    elif corr_type == 'Cos':
        ylabel = "Cosine similarity"
    else:
        ylabel = "Pearson's "+r"$\mathit{r}$"
    axes.set_xlabel('CNN layer', font=fpath)
    axes.set_ylabel(ylabel, font=fpath)
    plt.show()

# PCA3

In [ ]:
# Load CNN RDM
cdd_n, epoc = 5, 56
depth = 15
cnnrdms = np.zeros((sbjs_num,depth,16,16))
for li in range(depth):
    for sbj_idx in range(sbjs_num):
        filename = '_ind192to16_90_P{}_cdd0{}_L{}.npz'.format(str(sbj_list[sbj_idx]),str(cdd_n),str(li+1).zfill(2))
        savepath = base + '/RDMs/CNN/PCA/16x16/' + filename
        cnnrdms[sbj_idx,li] = np.load(savepath)['rdm']

# Cosine Similarity

In [ ]:
stim_type_list_summ = ['dig','obj','offd','mag','ani']
tt_list = ['Digit recognition','Object recognition','Digit vs. Object','Magnitude of digits','Animacy of objects']

In [ ]:
from numpy.linalg import norm
def cos_sim(A, B):
       return np.dot(A, B)/(norm(A)*norm(B))

In [ ]:
def cnn_rdm_cos2(cnnrdms,stim_type):
    cnnrdm = cnnrdms[sbj_idx]
    summ_arr = np.zeros((depth))
    il = np.tril_indices(16,-1)
    for li, layrdm in enumerate(cnnrdm):
        summ_arr[li] = cos_sim(cd[il],rdm_reord(layrdm)[il])
    return summ_arr

In [ ]:
from pathlib import Path
import matplotlib as mpl
fpath = Path(mpl.get_data_path(), "/users/jmy/data/image_sound/arial.ttf")

In [ ]:
for ci,cd in enumerate(cds_list):
    grp = np.zeros((sbjs_num, depth))
    type_, stim_type = interest_list[ci],stim_type_list_summ[ci]
    for sbj_idx in range(sbjs_num):
        grp[sbj_idx] = cnn_rdm_cos2(cnnrdms, stim_type)
    plot_corr_bar(grp,corr_type='Cos',std=True)